In [2]:
import SimpleITK as sitk
import os

file_name = "OSMSC0069-cm"
file_name = "OSMSC0070-cm"
file_name = "Angio_z_powrotem_Flash_20211129105819_6"
#file_name = "OSMSC00101-cm"
#file_name = "OSMSC0102-cm"
#file_name = "OSMSC0103-cm"
#file_name = "OSMSC0104-cm"
#file_name = "OSMSC0105-cm"
#file_name = "OSMSC0106-cm"
#file_name = "OSMSC0107-cm"
#file_name = "OSMSC0111-cm"

In [ ]:
# vti file test

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./repository/0070_0001/Images/OSMSC0070-cm.vti --pipe\
        vmtkrenderer --pipe vmtkimageviewer"

res = pypes.PypeRun(args)

In [ ]:
# nii.gz file test

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./data/nifti/OSMSC0070-cm.nii.gz --pipe\
        vmtkrenderer --pipe vmtkimageviewer"

res = pypes.PypeRun(args)

In [ ]:
# nii.gz file test

from vmtk import pypes
from vmtk import vmtkscripts

args = "vmtkimagereader -ifile ./data/nifti/Angio_z_powrotem_Flash_20211129105819_6.nii.gz --pipe\
        vmtkrenderer --pipe vmtkimageviewer"

res = pypes.PypeRun(args)

In [ ]:
# Convert Vascular model database VTI files to NIFTI as required by the segmentator we are using

from vmtk import pypes
from vmtk import vmtkscripts

for directory in os.listdir("./repository/"):
    print(directory)
    path = "./repository/" + directory + "/Images/";
    if os.path.isdir(path):
        for file in os.listdir(path):
            in_path = path + file
            out_path = "./data/nifti/" + file.replace(".vti", ".nii.gz")
            args = "vmtkimagereader -ifile " + in_path + " --pipe vmtkimagewriter -ofile " + out_path
            print(out_path)
            res = pypes.PypeRun(args)

In [4]:
# TotalSegmentator usage test

from totalsegmentator.python_api import totalsegmentator as ts

indir = "./data/nifti/" + file_name + ".nii.gz"
outdir = "./segmentations/" + file_name + "/"

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

ts(indir, outdir)

./data/nifti/Angio_z_powrotem_Flash_20211129105819_6.nii.gz

If you use this tool please cite: https://doi.org/10.48550/arXiv.2208.05868

Resampling...
  Resampled in 1.44s
Predicting part 1 of 5 ...
Predicting part 2 of 5 ...
Predicting part 3 of 5 ...
Predicting part 4 of 5 ...
Predicting part 5 of 5 ...
  Predicted in 29.24s
Resampling...
Saving segmentations...


  0%|          | 0/104 [00:00<?, ?it/s]

  Saved in 17.00s


In [ ]:
# Process and format segmented files for visualization

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)

for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file, outputPixelType = sitk.sitkInt32)

        path = outdir + file.replace(".nii.gz", ".mhd")
        mask_filter = sitk.MultiplyImageFilter()
        res = mask_filter.Execute(main, img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

In [5]:
# More lightweight solution for just the layer mask

indir = "./segmentations/" + file_name + "/";
outdir = "./segmentations/" + file_name + "/mhd/";

print(indir)

if not os.path.exists(outdir) :
    os.makedirs(outdir)
    
for file in os.listdir(indir):
    if os.path.isfile(indir + file) and ".nii.gz" in file :
        img = sitk.ReadImage(indir + file)
        
        path = outdir + file.replace(".nii.gz", ".mhd")
        scale_filter = sitk.ShiftScaleImageFilter()
        scale_filter.SetScale(100)
        res = scale_filter.Execute(img)

        minmax_filter = sitk.MinimumMaximumImageFilter()
        minmax_filter.Execute(img)
        
        img.SetSpacing([1, 1, 1])
        
        if minmax_filter.GetMaximum() != 0 :
            sitk.WriteImage(res, path)

./segmentations/Angio_z_powrotem_Flash_20211129105819_6/


In [ ]:
in_path = "./data/nifti/" + file_name + ".nii.gz"
out_path = "./data/mhd/" + file_name + ".mhd"

img = sitk.ReadImage(in_path)

print(img.GetSpacing())

nda = sitk.GetArrayFromImage(img)
print(img.GetSize())

img.SetSpacing([1, 1, 1])

print(img.GetSpacing())

sitk.WriteImage(img, out_path)

print(out_path)

In [ ]:
# Create MIP renderer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

importer = fast.ImageFileImporter.create("./data/mhd/" + file_name + ".mhd")
renderer = fast.MaximumIntensityProjection.create()
renderer.connect(importer)
window.connect(renderer)

# Run everything!
window.run()

In [ ]:
# Create volume renderer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

importer = fast.ImageFileImporter.create("./data/mhd/" + file_name + ".mhd")
renderer = fast.AlphaBlendingVolumeRenderer.create()
renderer.connect(importer)
window.connect(renderer)


# Run everything!
window.run()

In [3]:
# Create visualizer

import platform
if platform.system() != 'Windows':
    import PySide2.QtSvg # Must import this before fast due to conflicting symbols
import fast # Must import FAST before rest of pyside2
from PySide2.QtWidgets import *
from PySide2.QtCore import Qt
from shiboken2 import wrapInstance
import random as rd
import functools
import keyboard

segdir = "./segmentations/" + file_name + "/mhd/"

renderers = []
active = []

def import_segment(x) :
    print(x)
    
    importer = fast.ImageFileImporter.create(segdir + x)

    extraction = fast.SurfaceExtraction\
            .create(threshold=50)\
            .connect(importer)

    r = rd.uniform(0.5, 1);
    g = rd.uniform(0.5, 1);
    b = rd.uniform(0.5, 1);

    mesh = extraction.runAndGetOutputData()

    renderer = fast.TriangleRenderer\
            .create(color=fast.Color(r, g, b))\
            .connect(mesh)
    
    renderers.append(renderer)
    active.append(True)

    window.connect(renderer)
    
def toggle_segment_alpha(i):
    print(i)
    window.removeAllRenderers()
    window.connect(renderers[i])
    active[i] = True
    renderers[i].setOpacity(0, 1)
    
    changed = []
    
    for j in range(0, len(renderers)):
        if i != j and active[j]:
            window.connect(renderers[j])
            if not keyboard.is_pressed('q'):
                active[j] = False
                renderers[j].setOpacity(0, 0.05) 
                changed.append(j)
    
    for j in range(0, len(renderers)):
        if i != j and not active[j] and j not in changed:
            window.connect(renderers[j])
            renderers[j].setOpacity(0, 0.05)    
            
def reset_segment_alpha():
    for j in range(0, len(renderers)):
        renderers[j].setOpacity(0, 1)
        active[j] = True
            
    
def toggle_segment(i):
    return lambda: toggle_segment_alpha(i)

def export_mesh():
    return 0

# Create FAST Pipeline and window
window = fast.SimpleWindow3D.create(width=1024, height=512)\

# Get the underlying QtWidget of the FAST window and convert it to pyside2
mainWidget = wrapInstance(int(window.getWidget()), QWidget)

# Create GUI in Qt
layout = mainWidget.layout()
menuWidget = QWidget()
layout.addWidget(menuWidget)
menuLayout = QVBoxLayout()
menuWidget.setLayout(menuLayout)
menuLayout.setAlignment(Qt.AlignTop)
menuWidget.setFixedWidth(400)

# Create segment list
widget = QWidget()   

seglist = QVBoxLayout()
seglist.setAlignment(Qt.AlignTop)

scroll = QScrollArea()
scroll.setVerticalScrollBarPolicy(Qt.ScrollBarAlwaysOn)
scroll.setHorizontalScrollBarPolicy(Qt.ScrollBarAlwaysOff)
scroll.setWidgetResizable(True)

files = os.listdir(segdir)

idx = 0

for i in range(0, len(files)) :
    if os.path.isfile(segdir + files[i]) and ".mhd" in files[i] :
        elem_button = QPushButton(files[i][:-4])
        elem_button.clicked.connect(toggle_segment(idx))
        import_segment(files[i])
        seglist.addWidget(elem_button)
        idx += 1
        
widget.setLayout(seglist)
scroll.setWidget(widget)
        
reset = QPushButton("Reset view")
reset.clicked.connect(lambda: reset_segment_alpha())
    
menuLayout.addWidget(reset)
menuLayout.addWidget(scroll)

reset = QPushButton(files[i][:-4])
        
# Run everything!
window.run()

aorta.mhd
autochthon_left.mhd
autochthon_right.mhd
clavicula_left.mhd
clavicula_right.mhd
esophagus.mhd
heart_atrium_left.mhd
heart_atrium_right.mhd
heart_myocardium.mhd
heart_ventricle_left.mhd
heart_ventricle_right.mhd
humerus_left.mhd
humerus_right.mhd
inferior_vena_cava.mhd
kidney_right.mhd
liver.mhd
lung_lower_lobe_left.mhd
lung_lower_lobe_right.mhd
lung_middle_lobe_right.mhd
lung_upper_lobe_left.mhd
lung_upper_lobe_right.mhd
pulmonary_artery.mhd
rib_left_1.mhd
rib_left_2.mhd
rib_left_3.mhd
rib_left_4.mhd
rib_left_5.mhd
rib_left_6.mhd
rib_left_7.mhd
rib_left_8.mhd
rib_left_9.mhd
rib_right_1.mhd
rib_right_10.mhd
rib_right_2.mhd
rib_right_3.mhd
rib_right_4.mhd
rib_right_5.mhd
rib_right_6.mhd
rib_right_7.mhd
rib_right_8.mhd
rib_right_9.mhd
scapula_left.mhd
scapula_right.mhd
spleen.mhd
stomach.mhd
trachea.mhd
vertebrae_C7.mhd
vertebrae_T1.mhd
vertebrae_T10.mhd
vertebrae_T11.mhd
vertebrae_T2.mhd
vertebrae_T3.mhd
vertebrae_T4.mhd
vertebrae_T5.mhd
vertebrae_T6.mhd
vertebrae_T7.mhd
verteb